In [ ]:
%pushd ../../

In [ ]:
%env CUDA_VISIBLE_DEVICES=2

In [ ]:
import json

import os
import sys
import tempfile
from tqdm.auto import tqdm

import torch
import torchvision
from torchvision import transforms
from PIL import Image
import numpy as np

torch.cuda.set_device(0)

In [ ]:
from netdissect import setting

In [ ]:
segopts = 'netpqc'

In [ ]:
segmodel, seglabels, _ = setting.load_segmenter(segopts)

In [ ]:
segmodel.get_label_and_category_names()

In [ ]:
!ls notebooks/stats/churches

In [ ]:
import glob

In [ ]:
ns = []
for f in glob.glob('/data/vision/torralba/ganprojects/placesgan/tracer/utils/samples/domes/*.png'):
    ns.append(int(os.path.split(f)[1][6:][:-4]))

In [ ]:
ns = sorted(ns)

In [ ]:
label2idx = {l: i for i, l in enumerate(seglabels)}

In [ ]:
sorted(seglabels)

In [ ]:
label2idx['dome']

In [ ]:
label2idx['building']

In [ ]:
label2idx['tree']

In [ ]:
class Dataset():
    def __init__(self, before, before_prefix, after, after_prefix, device='cpu'):
        self.before = before
        self.before_prefix = before_prefix
        self.after = after
        self.after_prefix = after_prefix
        self.device = device
        
    def __getitem__(self, key):
        before_seg = torch.load(os.path.join(self.before, f'{self.before_prefix}{key}.pth'), map_location=self.device)
        after_seg = torch.load(os.path.join(self.after, f'{self.after_prefix}{key}.pth'), map_location=self.device)
        mapped = after_seg.permute(1, 2, 0)[(before_seg == 1708).sum(0).nonzero(as_tuple=True)]
        assert mapped.shape[1] == 6
        return (mapped == 5).sum(), mapped.shape[0]

class Sampler(torch.utils.data.Sampler):
    def __init__(self, indices):
        self.indices = indices
        
    def __len__(self):
        return len(self.indices)
    
    def __iter__(self):
        yield from self.indices

In [ ]:
(before_segs == 5).sum(dim=(0, 2,3))

In [ ]:
def compute(before, before_pref, after, after_pref, tgt=5, tgtc=0, src=1708, srcc=2, ns=ns):
    total = 0
    count = 0

    import time

    for subn in tqdm(torch.as_tensor(ns).split(100)):
        t0 = time.time()
        before_segs = [
            torch.load(os.path.join(before, f'{before_pref}{n}.pth'), map_location='cpu') for n in subn]
        after_segs = [
            torch.load(os.path.join(after, f'{after_pref}{n}.pth'), map_location='cpu') for n in subn]
        t1 = time.time()
        before_segs = torch.stack(before_segs).cuda()
        after_segs = torch.stack(after_segs).cuda()
        mapped = after_segs[:, tgtc][before_segs[:, srcc] == src]
        t2 = time.time()
        total += (mapped == tgt).sum()
        count += mapped.shape[0]
        print(total, count, t1-t0,t2-t1)

    return total.item(), count

In [ ]:
dome2spire_overfit[0] / dome2spire_overfit[1]

In [ ]:
before = 'notebooks/stats/churches/domes'
before_pref = 'domes_'
after = 'notebooks/stats/churches/dome2spire/ours'
after_pref = 'dome2spire_'

dome2spire_ours = compute(before, before_pref, after, after_pref)

In [ ]:
before = 'notebooks/stats/churches/domes'
before_pref = 'domes_'
after = 'notebooks/stats/churches/dome2spire/overfit'
after_pref = 'image_'

dome2spire_overfit = compute(before, before_pref, after, after_pref)


In [ ]:
before = 'notebooks/stats/churches/church'
before_pref = 'church_'
after = 'notebooks/stats/churches/dome2spire_all/ours'
after_pref = 'dome2spire_all_'

dome2spire_all_ours = compute(before, before_pref, after, after_pref, ns=list(range(10000)))

In [ ]:
dome2spire_all_ours[0] / dome2spire_all_ours[1]

In [ ]:
dome2spire_all_overfit[0] / dome2spire_all_overfit[1]

In [ ]:
before = 'notebooks/stats/churches/church'
before_pref = 'church_'
after = 'notebooks/stats/churches/dome2spire_all/overfit'
after_pref = 'image_'

dome2spire_all_overfit = compute(before, before_pref, after, after_pref, ns=list(range(10000)))

In [ ]:
before = 'notebooks/stats/churches/domes'
before_pref = 'domes_'
after = 'notebooks/stats/churches/dome2spire/ours'
after_pref = 'dome2spire_'

all_mapped = []

total = 0
count = 0

import time

for subn in tqdm(torch.as_tensor(ns).split(100)):
    t0 = time.time()
    before_segs = [
        torch.load(os.path.join(before, f'{before_pref}{n}.pth'), map_location='cpu') for n in subn]
    after_segs = [
        torch.load(os.path.join(after, f'{after_pref}{n}.pth'), map_location='cpu') for n in subn]
    t1 = time.time()
    before_segs = torch.stack(before_segs).cuda()
    after_segs = torch.stack(after_segs).cuda()
#     mapped = after_segs.permute(0, 2, 3, 1)[before_segs[:, 2] == 1708]
    mapped = after_segs[:, 0][before_segs[:, 2] == 1708]
#     all_mapped.append()
    t2 = time.time()
    total += (mapped == 5).sum()
    count += mapped.shape[0]
    print(total, count, t1-t0,t2-t1)


In [ ]:
before = 'notebooks/stats/churches/domes'
before_pref = 'domes_'
after = 'notebooks/stats/churches/dome2spire/ours'
after_pref = 'dome2spire_'

dataset = Dataset(before, before_pref, after, after_pref)

def wif(*args):
    torch.set_num_threads(8)
    
def cfn(l):
    return torch.stack([p[0] for p in l]).sum(), sum(p[1] for p in l)
    

loader = torch.utils.data.DataLoader(dataset, num_workers=10, batch_size=50, sampler=Sampler(ns), collate_fn=cfn, worker_init_fn=wif)

all_mapped = []

for mapped in tqdm(loader):
    all_mapped.append(mapped)


In [ ]:
after_seg.permute(1, 2, 0)[(before_seg == 1708).to(torch.int64).sum(0).nonzero(as_tuple=True)].shape

In [ ]:
!ls notebooks/stats/churches/dome2spire/ours

In [ ]:
class UnsupervisedImageFolder(torchvision.datasets.ImageFolder):
    def __init__(self, root, transform=None, max_size=None, get_path=False):
        self.temp_dir = tempfile.TemporaryDirectory()
        os.symlink(root, os.path.join(self.temp_dir.name, 'dummy'))
        root = self.temp_dir.name
        super().__init__(root, transform=transform)
        self.get_path = get_path
        self.perm = None
        if max_size is not None:
            actual_size = super().__len__()
            if actual_size > max_size:
                self.perm = torch.randperm(actual_size)[:max_size].clone()
                logging.info(f"{root} has {actual_size} images, downsample to {max_size}")
            else:
                logging.info(f"{root} has {actual_size} images <= max_size={max_size}")

    def _find_classes(self, dir):
        return ['./dummy'], {'./dummy': 0}

    def __getitem__(self, key):
        if self.perm is not None:
            key = self.perm[key].item()
        sample = super().__getitem__(key)[0]
        if self.get_path:
            path, _ = self.samples[key]
            return sample, path
        else:
            return sample
            

    def __len__(self):
        if self.perm is not None:
            return self.perm.size(0)
        else:
            return super().__len__()

In [ ]:
len(seglabels)

In [ ]:
transform = transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])


In [ ]:
def process(img_path, seg_path, device='cuda', batch_size=128, **kwargs):
    os.makedirs(seg_path, exist_ok=True)

    dataset = UnsupervisedImageFolder(img_path, transform=transform, get_path=True)
    loader = torch.utils.data.DataLoader(dataset, num_workers=24, batch_size=batch_size, pin_memory=True)  
    
    with torch.no_grad():
        for x, paths in tqdm(loader):
            segs = segmodel.segment_batch(x.to(device), **kwargs).detach().cpu()
            for path, seg in zip(paths, segs):
                k = os.path.splitext(os.path.basename(path))[0]
                torch.save(seg, os.path.join(seg_path, k + '.pth'))
            del segs

In [ ]:
import glob

In [ ]:
torch.backends.cudnn.benchmark=True

In [ ]:
process(
    '/data/vision/torralba/ganprojects/placesgan/tracer/utils/samples/domes',
    'churches/domes',
    batch_size=12)

In [ ]:
process(
    '/data/vision/torralba/ganprojects/placesgan/tracer/utils/samples/dome2tree',
    'churches/dome2tree/ours',
    batch_size=8)

In [ ]:
process(
    '/data/vision/torralba/ganprojects/placesgan/tracer/utils/samples/dome2spire',
    'churches/dome2spire/ours',
    batch_size=8)